In [ ]:
from pyspark.sql import SparkSession
# Initialize Spark Session
spark = SparkSession. builder \
.appName ("ReduceByKey_vs_GroupByKey") \
.getOrCreate ()

In [1]:
!hadoop fs -ls /tmp/

Found 9 items
drwxr-xr-x   - root       hadoop          0 2025-01-30 09:15 /tmp/active_cities.csv
-rw-r--r--   2 root       hadoop  343317147 2025-01-20 10:42 /tmp/customers.csv
-rw-r--r--   2 root       hadoop   10528211 2025-01-20 10:31 /tmp/customers_10mb.csv
-rw-r--r--   2 mayank0953 hadoop    1060750 2025-01-20 10:23 /tmp/customers_1mb.csv
-rw-r--r--   2 mayank0953 hadoop  570783961 2025-01-22 17:12 /tmp/customers_500mb.csv
drwxr-xr-x   - anonymous  hadoop          0 2025-01-23 16:41 /tmp/external
drwxrwxrwt   - hdfs       hadoop          0 2025-01-20 09:57 /tmp/hadoop-yarn
drwx-wx-wx   - hive       hadoop          0 2025-01-20 09:57 /tmp/hive
-rw-r--r--   2 root       hadoop         83 2025-01-26 19:11 /tmp/inputhdfsdbz.txt


In [6]:
!hadoop fs -head /tmp/customers_1mb.csv

customer_id,name,city,state,country,registration_date,is_active
0,Customer_0,Pune,Maharashtra,India,2023-06-29,False
1,Customer_1,Bangalore,Tamil Nadu,India,2023-12-07,True
2,Customer_2,Hyderabad,Gujarat,India,2023-10-27,True
3,Customer_3,Bangalore,Karnataka,India,2023-10-17,False
4,Customer_4,Ahmedabad,Karnataka,India,2023-03-14,False
5,Customer_5,Hyderabad,Karnataka,India,2023-07-28,False
6,Customer_6,Pune,Delhi,India,2023-08-29,False
7,Customer_7,Ahmedabad,West Bengal,India,2023-12-28,True
8,Customer_8,Pune,Karnataka,India,2023-06-22,True
9,Customer_9,Mumbai,Telangana,India,2023-01-05,True
10,Customer_10,Pune,Gujarat,India,2023-08-05,True
11,Customer_11,Delhi,West Bengal,India,2023-08-02,False
12,Customer_12,Chennai,Gujarat,India,2023-11-21,False
13,Customer_13,Chennai,Karnataka,India,2023-11-06,True
14,Customer_14,Hyderabad,Tamil Nadu,India,2023-02-07,False
15,Customer_15,Mumbai,Gujarat,India,2023-03-02,True
16,Customer_16,Chennai,Karnataka,India,2023-04-05,False
17,Customer_17,Hyd

In [2]:
hdfs_path = '/tmp/customers_1mb.csv'
rdd = spark.sparkContext.textFile(hdfs_path)

In [7]:
header = rdd.first()

In [8]:
rdd_no_header = rdd.filter(lambda row:row!=header).map(lambda row:row.split(','))

In [9]:
rdd_no_header.first()

['0', 'Customer_0', 'Pune', 'Maharashtra', 'India', '2023-06-29', 'False']

In [11]:
reduced_by_rdd = rdd_no_header.map(lambda row:(row[2],1)).reduceByKey(lambda x,y:x+y)

In [12]:
reduced_by_rdd.collect()

[('Mumbai', 2142),
 ('Chennai', 2194),
 ('Pune', 2243),
 ('Bangalore', 2211),
 ('Hyderabad', 2242),
 ('Ahmedabad', 2198),
 ('Delhi', 2200),
 ('Kolkata', 2223)]

In [ ]:
# Data written is 472 Bytes

In [13]:
grouped_by_rdd  = rdd_no_header.map(lambda row:(row[2],1)).groupByKey()

In [14]:
grouped_by_rdd.collect()

[('Mumbai', <pyspark.resultiterable.ResultIterable at 0x7f909e6bbaf0>),
 ('Chennai', <pyspark.resultiterable.ResultIterable at 0x7f909e9748b0>),
 ('Pune', <pyspark.resultiterable.ResultIterable at 0x7f909e9752d0>),
 ('Bangalore', <pyspark.resultiterable.ResultIterable at 0x7f909e9755a0>),
 ('Hyderabad', <pyspark.resultiterable.ResultIterable at 0x7f909e974700>),
 ('Ahmedabad', <pyspark.resultiterable.ResultIterable at 0x7f909e975120>),
 ('Delhi', <pyspark.resultiterable.ResultIterable at 0x7f909e9756f0>),
 ('Kolkata', <pyspark.resultiterable.ResultIterable at 0x7f909e975330>)]

In [16]:
grouped_by_result = grouped_by_rdd.map(lambda row:(row[0],len(row[1])))

In [17]:
grouped_by_result.collect()

[('Mumbai', 2142),
 ('Chennai', 2194),
 ('Pune', 2243),
 ('Bangalore', 2211),
 ('Hyderabad', 2242),
 ('Ahmedabad', 2198),
 ('Delhi', 2200),
 ('Kolkata', 2223)]

In [18]:
hdfs_path = '/tmp/customers_500mb.csv'
rdd = spark.sparkContext.textFile(hdfs_path)

In [ ]:
#Reduce By Key - 1 shot

In [19]:
rdd.map(lambda row:row.split(',')).map(lambda row:(row[2],1)).reduceByKey(lambda x,y:x+y).collect()

[('Chennai', 1095052),
 ('Pune', 1095748),
 ('Kolkata', 1096777),
 ('Hyderabad', 1096426),
 ('Ahmedabad', 1097162),
 ('Mumbai', 1095815),
 ('city', 1),
 ('Delhi', 1096183),
 ('Bangalore', 1094195)]

In [ ]:
#GroupByKey - 1 shot

In [20]:
rdd.map(lambda row:row.split(',')).map(lambda row:(row[2],1)).groupByKey().map(lambda row:(row[0],len(row[1]))).collect()

[('Chennai', 1095052),
 ('Pune', 1095748),
 ('Kolkata', 1096777),
 ('Hyderabad', 1096426),
 ('Ahmedabad', 1097162),
 ('Mumbai', 1095815),
 ('city', 1),
 ('Delhi', 1096183),
 ('Bangalore', 1094195)]